In [45]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from rsdl.layers import Linear
from rsdl.optim import Adam
from rsdl.activations import Relu, Softmax
from rsdl import Tensor
import numpy as np

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_set = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_set = datasets.MNIST(
    root='./data',
    train=False,
    download=False,
    transform=ToTensor()
)

In [47]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [48]:
class Model:
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.layer0 = Linear(input_size, hidden_size, need_bias=True)
        self.layer1 = Linear(hidden_size, hidden_size, need_bias=True)
        self.layer2 = Linear(hidden_size, hidden_size, need_bias=True)
        self.layer3 = Linear(hidden_size, hidden_size, need_bias=True)
        self.layer4 = Linear(hidden_size, hidden_size, need_bias=True)
        self.layer5 = Linear(hidden_size, output_size, need_bias=True)
        self.optimizer =  Adam(layers=[self.layer0, self.layer1, self.layer2, self.layer3, self.layer4, self.layer5])
    
    def forward(self, x):
        # Assuming x is a tensor with size (batch_size, 1, 28, 28)
        x = Relu(self.layer0(x))
        x = Relu(self.layer1(x))
        x = Relu(self.layer2(x))
        x = Relu(self.layer3(x))
        x = Relu(self.layer4(x))
        x = Softmax(self.layer5(x))
        return x

In [49]:
model = Model(input_size=784, hidden_size=100, output_size=10)

In [50]:
from rsdl.losses import cross_entropy, mean_square_errors

for epoch in range(50):

    epoch_loss = 0.0
    batch = 0
    
    for images, labels in train_loader:
        batch += 1

        flattened_images = images.view([images.size(0), -1])
        numpy_images = flattened_images.numpy()
        inputs = Tensor(numpy_images)

        predicted = model.forward(inputs)

        actual = Tensor(np.zeros((len(labels), 10)))
        actual.data[np.arange(len(labels)), labels] = 1

        loss = mean_square_errors(predicted, actual)

        loss.backward()
        epoch_loss += loss.data

        model.optimizer.step()
        model.optimizer.zero_grad()

TypeError: unsupported operand type(s) for /: 'Tensor' and 'int'